# Dependencies

In [1]:
! pip -q install datasets evaluate torch nltk rank_bm25
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Enable GPU

In [3]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

# Generate Summary

In [13]:
from transformers import BertTokenizer, GPT2Tokenizer, EncoderDecoderModel

model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2gpt2-cnn_dailymail-fp16").to(device)
bert_tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.


In [14]:
query = "what does a metabolic acidosis need to reverse the condition"

In [15]:
article = """Metabolic acidosis is a condition characterized by an accumulation of acid in the body due to factors such as increased production of acid, decreased excretion of acid,
or loss of bicarbonate (a base) from the body. Reversing metabolic acidosis typically involves addressing the underlying cause and restoring the acid-base balance in the body.
The specific treatment depends on the underlying etiology of the metabolic acidosis. Here are some general approaches:

Correcting the Underlying Cause:
Identify and treat the underlying condition that is causing the metabolic acidosis. This may involve addressing issues such as kidney dysfunction, diabetes, lactic acidosis, or poisoning.

Fluid Replacement:
In some cases, intravenous fluids may be administered to help correct dehydration or restore electrolyte balance. The choice of fluids depends on the specific needs of the individual.

Electrolyte Replacement:
If there is an electrolyte imbalance contributing to the acidosis, such as low levels of potassium or sodium, the appropriate electrolytes may be replaced through medications or intravenous administration.

Bicarbonate Administration:
In certain situations where the acidosis is severe and the pH is very low, healthcare providers may administer sodium bicarbonate to help neutralize excess acid and raise the pH. However, the use of bicarbonate is not without risks, and its administration is carefully considered based on the specific circumstances.

Treatment of Underlying Diseases:
Manage and treat any underlying medical conditions contributing to the acidosis, such as diabetes, kidney disease, or sepsis.

It's crucial to note that the treatment of metabolic acidosis should be individualized based on the specific circumstances of the patient, and medical professionals,
such as physicians and nephrologists, play a key role in determining the appropriate course of action. Self-diagnosis and self-treatment are not recommended, and individuals experiencing symptoms of metabolic acidosis should seek prompt medical attention."""

In [20]:
input_ids = bert_tokenizer(article, return_tensors="pt").input_ids.to(device)
gen_tokens = model.generate(input_ids, max_length=2048,).to(device)
gen_summary = gpt2_tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [22]:
gen_summary = """Metabolic acidosis is a condition marked by an excess of acid in the body, resulting from various causes. To reverse this condition, the underlying cause must be identified and treated.
General approaches include correcting the primary issue, administering intravenous fluids and electrolytes, and, in severe cases, using sodium bicarbonate to neutralize excess acid.
Individualized treatment by healthcare professionals is crucial, as it depends on the specific circumstances of the patient, such as underlying diseases.
Self-diagnosis and treatment are discouraged, and individuals should seek prompt medical attention for symptoms of metabolic acidosis.
"""

# Generate prompts

In [23]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("satyanshu404/bart-large-cnn-prompt_generation")
seq_model = AutoModelForSeq2SeqLM.from_pretrained("satyanshu404/bart-large-cnn-prompt_generation").to(device)

In [24]:
def add_instruction(text):
  instruction = """Instruction: Based on the given description generate a list of all the prompts that are relavent to the document.
  Query: """
  return f"{instruction} {text}"

In [25]:
def generate_results(prompts, model, tokenizer):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
  gen_tokens = model.generate(input_ids).to(device)
  gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
  return gen_text

In [36]:
import time

start = time.time()
# extracting each sentence from each summary
sentences = gen_summary.split('\n')
queries = []
# enumerating through each summary
for sentence in sentences:
    # coverting into prompts
    prompt = add_instruction(sentence)
    # generating queries
    gen_text = generate_results(prompt, seq_model, tokenizer)
    # getting all the queries
    queries.extend(gen_text.split('?'))

end = time.time()

In [37]:
print("time:", end - start)

time: 10.214365482330322


In [38]:
queries

['Metabolic acidosis is a condition marked by an excess of acid in the body, resulting from various causes. To reverse this condition, the underlying cause must be identified and treated. What is the most common cause of metabolic acidosis',
 ' What are the main symptoms of the condition',
 '',
 'What is the primary issue',
 ' What is the best way to treat it',
 ' What are the best ways to treat acid reflux',
 ' How do you treat it with intravenous fluids and electrolytes',
 ' How can sodium bicarbonate be used to neutralize excess acid',
 ' How long does it take for the acid to clear up',
 '',
 'What are the most important questions to ask a patient',
 " What is the best way to treat a patient's underlying diseases",
 ' What are the best ways to treat an individualized treatment for a specific condition',
 ' How should a patient be treated for an underlying disease',
 ' How can a doctor help a patient get the best care for an individual',
 ', What are some of the most common symptoms 

# Top 10 prompts

In [42]:
from rank_bm25 import BM25Okapi

corpus = queries
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
prompts = bm25.get_top_n(tokenized_query, corpus, n=10)

In [43]:
doc_scores

array([6.37742902, 3.10319254, 0.        , 0.81474368, 1.32999138,
       1.27201113, 0.        , 0.60833879, 3.94751667, 0.        ,
       2.5900531 , 2.38233951, 3.83109598, 1.26298291, 2.14214815,
       0.61053616, 0.        , 3.54592579, 0.73301882, 0.66619454,
       0.69549876, 0.        , 0.63715215, 0.73038062, 0.93244437,
       0.61053616, 0.81189026, 0.        , 0.        ])

In [44]:
prompts

['Metabolic acidosis is a condition marked by an excess of acid in the body, resulting from various causes. To reverse this condition, the underlying cause must be identified and treated. What is the most common cause of metabolic acidosis',
 ' How long does it take for the acid to clear up',
 ' What are the best ways to treat an individualized treatment for a specific condition',
 'Self-diagnosis and treatment are discouraged, and individuals should seek prompt medical attention for symptoms of metabolic acidosis. What are the symptoms of acidosis',
 ' What are the main symptoms of the condition',
 'What are the most important questions to ask a patient',
 " What is the best way to treat a patient's underlying diseases",
 ' How can a doctor help a patient get the best care for an individual',
 ' What is the best way to treat it',
 ' What are the best ways to treat acid reflux']